# CS-6570 Lecture 19 - Multiple Hypothesis Testing
**Dylan Zwick**

*Weber State University*

In our last lecture we reviewed the basic procedures of hypothesis testing, and introduced some of the problems that can be encountered with multiple hypothesis testing. Today, we'll get into multiple hypothesis testing in greater depth, discussing some useful theoretical and practical techniques for limiting Type I error while not destroying the power (ability to avoid Type II error) of the test.

First, let's review what multiple hypothesis testing is. Suppose you're a genomic researcher and you sequence the genomes of people with and without some particular medical condition, and then, for each of 20,000 genes, test whether sequence variants in that gene are associated with the medical condition of interest. This amounts to performing $m = 20,000$ hypothesis tests. The analysis is exploratory, not confirmatory, in nature, in the sense that you do not have any particular hypothesis in mind, and instead want to see whether there is modest evidence for the association between each gene and the disease, with a plan to further investigate any genes for which there is such evidence. It's OK to get some false positives here, but a $p$-value of, say, $.05$ probably isn't going to work well for you either, as you'd expect by random chance to have $1,000$ genes to investigate. You will get a lot of Type-I errors and wrongly rejected the null hypothesis.

In our lecture today we'll discuss ways to balance Type I and Type II errors when doing exploratory testing.

But first, let's get some of the usual library imports out of the way.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**The Family-Wise Error Rate**

In our last lecture, we discussed testing multiple hypotheses while controlling the probability of making at least one Type I error. Suppose we want to test $m$ null hypotheses, then we define the true positive, false positive, true negative, and false negative counts as follows:

Columns are "truth" aka reality. Rows represent our actions (reject vs do not reject). Do not reject is more appropriate than "accept". "Rejecting the null hypothesis" is the good news for most tests. R is the total number of rejections. m-R is the number we do not reject.

Type I error is when you discover something and you announce the world and later have to reject.
Type II error is when you don't pick up on something and you should have. The "Power" of a criteria is how often you get miss it - like a frequency of Type II error: S / (W+S). The idea perfect test has V and W at 1 and no U or S.

|  | $H_{0}$ is True | $H_{0}$ is False | Total |
| :--- | :---: | :---: | :---: |
| Reject $H_{0}$ | V | S | R |
| Do Not Reject $H_{0}$ | U | W | m-R |
| Total | $m_{0}$ | $m$-$m_{0}$ | $m$ |

Recall from the last lecture that the Type I error rate is the probability of rejecting $H_{0}$ if $H_{0}$ is true, and the **family-wise error rate** (FWER) is the probability of making *at least one* Type I error. If we make the (rather strong) assumption that the $m$ tests are independent then if all $m$ null hypotheses are true and we reject the null hypothesis if it's $p$-value is less than $\alpha$, then:


$FWER(\alpha) = 1 - (1-\alpha)^{m}$

If $m$ is reasonably large, then for even a small but not insignificant value of $\alpha$ (like $.01$) this number will be very close to $1$. Below is a chart of the family-wise error rate as a function of $m$, the number of hypotheses tested, for three different values of $\alpha$.

<center>
    <div>
        <img src="FWER for Different alphas.png" width="800"/>
    </div>
</center>

Certainly if $m = 20,000$ you're basically guaranteed a false positive.

In class last time we learned the **Bonferroni method** for controlling the FWER, which states that if you want a FWER less than $\alpha$, you only accept $p$-values less than $\displaystyle \frac{\alpha}{m}$. If $m = 20,000$ then $\alpha$ will be *very* small, and this is probably not OK, because there will likely be a great deal of Type II errors.

A less restrictive variant of the Bonferroni method is **Holm's method** (a.k.a. Holm's step-down procedure). Holm's method is summarized in the algorithm description below. A proof that this method controls the FWER is not too bad, but is a bit beyond our scope for this class.

**Holm's Method**

1. Specify $\alpha$ (0.05), the level at which to control the FWER.

2. Compute $p$-values, $p_{1},\ldots,p_{m}$, for the $m$ null hypotheses $H_{01},\ldots,H_{0m}$. For every null hypothesis compute the p-values

3. Order the $m$ $p$-values so that $p_{(1)} \leq p_{(2)} \leq \cdots \leq p_{(m)}$. << This is the interesting part

4. Define $\displaystyle L = min\left\{ j : p_{(j)} > \frac{\alpha}{m + 1 - j}\right\}$.

5. Reject all null hypotheses $H_{0j}$ for which $p_{j} < p_{(L)}$.

It's worth noting that the threshold we use to reject each null hypothesis, $p_{(L)}$ depends on the values of *all* $m$ of the $p$-values. Holm's method is uniformly more powerful than the Bonferroni method, and should always be preferred. You don't know your threshold until you've done some analysis on the data.

We'll investigate these methods using the [Fund](https://islp.readthedocs.io/en/latest/datasets/Fund.html) dataset provided for the book [An Introduction to Statistical Learning](https://www.statlearning.com/). It's a simulated data set containing the returns for 2,000 hedge fund managers. To load this dataset, we'll want to install the ISLP package, which we do below before we load the Fund dataset.

In [8]:
!pip install ISLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 21.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.2/522.2 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 21.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.7/350.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 24.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 14.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=189aa0310d17ae421ef0ce64c2f54c5b86e1bea176823450414f4c6e8d620c18
  Stored in directory: /Users/robchristiansen/Library/Caches/pip/wheels/8b/67/f4/2c

In [9]:
from ISLP import load_data
Fund = load_data('Fund')
Fund.head()

,Manager1,Manager2,Manager3,Manager4,Manager5,Manager6,Manager7,Manager8,Manager9,Manager10,...,Manager1991,Manager1992,Manager1993,Manager1994,Manager1995,Manager1996,Manager1997,Manager1998,Manager1999,Manager2000
0,-3.341992,-4.167469,9.389223,8.417220,0.997863,7.191473,-10.767592,4.072425,1.575264,-0.798505,...,-2.948706,10.350706,-2.855337,-4.431786,0.739544,0.198044,1.752188,-1.534710,-3.359419,6.585654
1,3.759627,12.525254,3.403366,0.143944,-7.222227,0.067747,-10.737053,-1.138185,-7.166604,4.778522,...,24.003150,-1.966606,-1.609109,1.405325,4.717175,1.540359,-12.218233,-0.073008,-8.547683,-2.382629
2,12.970091,-2.581061,-0.824734,6.584604,17.050241,1.857130,3.196942,-7.981362,-1.214148,2.338250,...,-2.926914,6.420147,8.946921,3.449013,1.009957,1.481369,14.203314,0.005562,-5.105035,2.292429
3,-4.874630,7.981743,-4.026743,-4.731946,0.503276,0.740187,-28.969410,4.683751,-0.568840,-4.000547,...,-3.112208,3.173581,-6.017109,-1.984873,1.022525,-2.261927,19.345970,-1.048299,-0.016154,1.196832
4,2.019279,-5.370236,-4.854669,10.594432,-6.891574,9.877838,1.430033,9.840311,5.311455,18.365094,...,7.173653,-9.157211,7.643125,-1.022339,-1.325865,2.848785,-6.642081,2.488612,0.032060,-7.510032


We can now constuct a one-sample $t$-test for each of the first five managers in this dataset, to test the null hypothesis that the $j$th fund manager's mean return equals zero, $H_{0,j}: \mu_{j} = 0$. But first, we'll want to import the one-sample $t$-test function from the stats library in scipy. This is a two-sided T-test.

In [10]:
from scipy.stats import ttest_1samp

In [13]:
fund_mini = Fund.iloc[:,:5]
fund_mini_pvals = np.empty(5) # This is a good pattern
for i in range(5):
    fund_mini_pvals[i] = ttest_1samp(fund_mini.iloc[:,i], 0).pvalue # We believe the average returns of fund managers is zero. 
fund_mini_pvals

array([0.00620236, 0.91827115, 0.01160098, 0.6005396 , 0.75578151])

The $p$-values are low for Managers One and Three, and high for the other three managers. However, we cannot simply reject $H_{0,1}$ and $H_{0,3}$, since this would fail to account for the multiple testing that we have performed. Instead, we will conduct Bonferroni’s method and Holm’s method to control the FWER.

To do this, we use the [multipletests](https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html) function from the [statsmodels](https://www.statsmodels.org/dev/index.html) module (abbreviated to *mult_test*). Given the $p$-values, for methods like Holm and Bonferroni the function outputs *adjusted p-values*, which can be thought of as a new set of $p$-values that have been corrected for multiple $p$-values testing. If the adjusted $p$-value for a given hypothesis is less than or equal to $\alpha$, then that hypothesis can be rejected while maintaining a FWER of no more than $\alpha$. In other words, for such methods, the adjusted $p$-values resulting from the *multipletests* function can simply be compared to the desired FWER in order to determine whether or not to reject each hypothesis. We will later see that we can use the same function to control FDR as well.

The *mult_test* function takes $p$-values and a method argument, as well as an optional $\alpha$ argument. It returns the decisions (reject below) as well as the adjusted $p$-values (bonf).

In [14]:
from statsmodels.stats.multitest import multipletests as mult_test

In [1]:
# Means that there is less than a 5% chance that we reject the null hypothesis when we not have rejected it. alpha is the family wise error rate.
reject , bonf = mult_test(fund_mini_pvals , method = "bonferroni", alpha = .05)[:2]

reject

NameError: name 'mult_test' is not defined

In [23]:
# Gives the adjusted p-values 
bonf

array([0.03101178, 1.        , 0.05800491, 1.        , 1.        ])

These are simply the *fund_mini* $p$-values multiplied by $5$ and truncated to be less than or equal to $1$.

So, using the Bonferroni method we were able to reject one null hypothesis, for the first fund manager. Fund manager 3 was awful close.

On the other hand, using Holm's method, we get:

In [24]:
mult_test(fund_mini_pvals , method = "holm", alpha = .05)[:2]

(array([ True, False,  True, False, False]),
 array([0.03101178, 1.        , 0.04640393, 1.        , 1.        ]))

So, we reject the null hypothesis both for manager 1 and manager 3. Holm's method has more *power* than Bonferroni's. Holm's method is likely to have Type II error (accepted when should have rejected (and published a celebratory article))

Using both Holm's method and the Bonferroni method, rejecting the null hypothesis becomes more difficult as the number of hypotheses, $m$, increases. This makes sense. The goal of the FWER is to prevent *any* Type I error. If there are many hypotheses, then to prevent any Type I error, you'll end up accepting the null hypothesis almost every time. This will significantly decrease the power of your method, and you'll likely miss cases where you should reject the null hypothesis. In practice, when $m$ is large, we may be willing to tolerate a few false positives, in the interest of making more discoveries. This is the motivation behind the **false discovery rate**. So, an FDR (usually denoted with $q$) of 20% would correspond with rejecting as many null hypotheses as possible while guaranteeing no more than 20% of those rejected are, on average, false positives.

**The False Discovery Rate**

When $m$ is large, then trying to reject *any* Type I error is just too stringent. Instead , we might try to make sure the ratio of false positives (V) to total positives (V + S = R) is sufficiently low, so that most of the rejected null hypotheses are not false positives. The ratio $\displaystyle \frac{V}{R}$ is knows as the *false discovery proportion* (FDP). V is the number of times you incorrectly reject the null hypothesis. R is the total number of rejections. Now, of course, in practice we don't know which positives are false and which are not. If we did, we wouldn't be doing experiments. So, we can't work with the FDP directly, and instead focus on its expectation $E(FDP)$ which is the *false discovery rate* (FDR). Note there is no standard, accepted threshold for FDR control. Instead, the choice of FDR threshold is typically context-dependent.

So, for a set value of $q$, what is the corresponding rejection criteria on the $p$-value? Well, it turns out there's a very simple procedure for calculating it, known as the *Benjamini-Hochberg* procedure.

1. Specify $q$, the level at which to control the FDR.

2. Computer $p$-values $p_{1},\ldots,p_{m}$, for the $m$ null hypotheses $H_{01},\ldots,H_{0m}$.

3. Order the $m$ $p$-values so that $p_{(1)} \leq p_{(2)} \leq \cdots \leq p_{(m)}$.

4. Define $\displaystyle L = max\left\{j : p_{(j)} \leq \frac{qj}{m}\right\}$.

5. Reject all null hypotheses $H_{0}$ for which $p_{j} \leq p_{(L)}$.

The proof of this is well beyond the scope of this class. Also, note the rejection threshold depends upon the $p$-values of our data, and so (unlike with Bonferroni) we can't know what it will be before the data is analyzed.

Going back to our *Fund* dataset, we can perform hypothesis tests for all 2,000 fund managers. We perform a one-sample $t$-test of $H_{0,j} : \mu_{j} = 0$, which states that the $j$th fund manager's mean return is zero.

In [25]:
fund_pvalues = np.empty(2000)
for i, manager in enumerate(Fund.columns):
    fund_pvalues[i] = ttest_1samp(Fund[manager], 0).pvalue

There are far too many managers to consider trying to control the FWER. Instead, we  focus on controlling the FDR: that is, the expected fraction of rejected null hypotheses that are actually false positives. The *multi-test* function we used earlier can be used to carry out the Benjaminni-Hochberg procedure described above.

In [26]:
fund_qvalues = mult_test(fund_pvalues , method = "fdr_bh")[1]
fund_qvalues [:10]

array([0.08988921, 0.991491  , 0.12211561, 0.92342997, 0.95603587,
       0.07513802, 0.0767015 , 0.07513802, 0.07513802, 0.07513802])

The $q$-values output can be interpreted as the smallest FDR threshold at which we would reject a particular null hypothesis. For example, a $q$-value of $0.1$ indicates that we can reject the corresponding null hypothesis at an FDR of 10% or greater, but that we cannot reject the null hypothesis at an FDR below 10%. If we control the FDR at 10%, how many fund managers can we reject?

In [27]:
(fund_qvalues <= 0.1).sum()

146

146 fund managers could be considered. On the other hand, if we'd used Bonferroni's method to control the FWER at level $\alpha = .1$, we would have failed to reject any null hypotheses! Bonferroni's method is very WEAK (you'll get a lot of Type II errors)

In [29]:
(fund_pvalues <= 0.1 / 2000).sum()

0

**A Re-Sampling Approach to $p$-Values and False Discovery Rates**

Let's say X represents a population with a medical condition and Y represents the population without a medical condition. We'll say the average value of X = the average of Y (null hypothesis). Furthermore, we'll say the populations are exactly the same. If the populations are indeed the same, then we should be able to swap samples between X and Y (permute the observations). We're going to take our observations and we're going to combine them into a new set of observations and then we randomly permute the observations into new X(1 star) and Y (1 star).x_{1}^*. Then we can get a T-statistic of how apart the means are.  

So far, we've assumed we're testing a particular null hypothesis $H_{0}$ using a test statistic $T$, which has some assumed distribution under $H_{0}$. However, if our null hypothesis $H_{0}$ or test statistic $T$ is somewhat unusual, then it may be the case that no theoretical null distribution is available. Alternatively, even if a theoretical null distribution exists, then we may be wary of relying upon it, perhaps because some assumption that is required for it to hold is violated. For example, maybe the sample size is small. We can still perform inference in this setting, exploiting the availability of fast computers.

Recall from our last lecture that that if our null hypothesis is the mean of a random variable $X$ equals the mean of a random variable $Y$, then given $n_{X}$ and $n_{Y}$ independent observations of $X$ and $Y$ respectively, the two-sample t-statistic is if the form:

$\displaystyle T = \frac{\hat{\mu_{X}} - \hat{\mu_{Y}}}{s\sqrt{\frac{1}{n_{X}} + \frac{1}{n_{Y}}}}$

where $\hat{\mu_{X}}$ and $\hat{\mu_{Y}}$ are the sample means for the test and control groups respectively, and

$\displaystyle s = \sqrt{\frac{(n_{X}-1)s_{Y}^{2} + (n_{Y}-1)s_{Y}^{2}}{n_{X} + n_{Y} - 2}}$

is an estimator of the pooled standard deviation of the two samples. Here, $\displaystyle s_{X}^{2} = \frac{\sum_{i = 1}^{n_{X}}(x_{i}^{2}-\mu_{X}^{2})}{n_{X}-1}$ and $\displaystyle s_{Y}^{2} = \frac{\sum_{i = 1}^{n_{Y}}(y_{i}^{2}-\mu_{Y}^{2})}{n_{Y}-1}$ are unbiased estimators of the variance of the blood pressure in the treatment and control groups, respectively. A large (absolute) value of $T$ provides evidence against $H_{0}: \mu_{X} = \mu_{Y}$, and hence evidence in support of $H_{a}: \mu_{X} \neq \mu_{Y}$.

If $n_{X}$ and $n_{Y}$ are large, then $T$ is approximately a normal distribution with mean 0 and variance 1. However, if, say, $n_{X}$ and $n_{Y}$ are small, then in the absence of a strong assumption about the distributions of $X$ and $Y$, we do not know the theoretical null distribution of $T$. In this case, we can approximate the null distribution of $T$ using a *re-sampling* approach, or more specifically, a *permutation* approach.

To see this, we conduct a thought experiment. If $H_{0}$ holds, so that $\mu_{X} = \mu_{Y}$, and we make the *stronger* assumption that the distributions of $X$ and $Y$ are the same, then the distribution of $T$ is invariant under swapping observations of $X$ with observations of $Y$. That is, if we randomly swap some of the observations in $X$ with the observations in $Y$, then the test statistic $T$ computed based on this swapped data has the same distribution as $T$ based on the original data. Please note this is true only if $H_{0}$ holds and the distributions of $X$ and $Y$ are the same.

This suggests the following approach. To approximate the null distribution of $T$, we randomly permute the $n_{X} + n_{Y}$ observations $B$ times, for some large value of $B$ like $10,000$, and each time we compute the $T$ statistic. We can let $T^{*1}, \ldots, T^{*B}$ denote the values of these $T$ statistics on the permuted data. Using these, to compute a $p$-value for $T$, we simply compute:

$p$-value $\displaystyle = \frac{\sum_{b = 1}^{B} 1_{(|T^{*b}| \geq |T|)}}{B}$

We can define this procedure algorithmically as:

1. Compute $T$ on the original data $x_{1},x_{2},\ldots,x_{n_{X}}$, and $y_{1},y_{2},\ldots,y_{n_{Y}}$.

2. For $b = 1,2,\ldots,B$, where $B$ is a large number (like $10,000$):

    (a) Permute the $n_{X}$ and $n_{Y}$ observations at random. Call the first $n_{X}$ permuted observations $x_{1}^{*},x_{2}^{*},\ldots,x_{n_{X}}^{*}$, and call the remaining observations $y_{1}^{*},y_{2}^{*},\ldots,y_{n_{Y}}^{*}$.
    
    (b) Compute $T$ on the permuted data $x_{1}^{*},x_{2}^{*},\ldots,x_{n_{X}}^{*}$, and $y_{1}^{*},y_{2}^{*},\ldots,y_{n_{Y}}^{*}$, and call the result $T^{*b}$.
    
3. The $p$-value is given by $\displaystyle \frac{\sum_{b = 1}^{B} 1_{(|T^{*b}| \geq |T|)}}{B}$

We can also apply this approach to the False Discovery Rate (FDR) calculation. Suppose we wish to control the FDR for $m$ null hypotheses, $H_{01},\ldots,H_{0m}$, where either no theoretical null distribution is available, or we prefer to avoid the use of a theoretical null distribution. We can make use of the two-sample $t$-statistic for each hypothesis, leading to the test statistics $T_{1},\ldots,T_{m}$. We could simply compute the $p$-value for each of the $m$ hypotheses using the re-sampling method above, and then apply the Benjamini-Hochberg procedure to these $p$-values. However, we can do this in an even more direct way without calculating $p$-values!

Recall the FDR is defined as $\displaystyle E\left(\frac{V}{R}\right)$ where $V$ is the count of false-positives, and $R$ is the count of true-positives. To estimate the FDR via re-sampling, we make the assumption that $\displaystyle E\left(\frac{V}{R}\right) \approx \frac{E(V)}{R}$. If we reject any null hypothesis for which the test statistic exceeds some number $c$ is absolute value, then calculating $R$ is straightforward, $\displaystyle R = \sum_{i = 1}^{m} 1_{(|T_{j}| \geq c)}$. However, estimating $V$ is difficult. One way to do it is with a re-sampling approach.

For each of the null hypotheses $H_{01},\ldots,H_{0m}$, we can estimate $E(V)$ as follows. Let $x_{1}^{(j)},\ldots,x_{n_{X}}^{(j)}$ and $y_{1}^{(j)},\ldots,y_{n_{Y}}^{(j)}$ be the data associated with the $j$th null hypothesis, with $j = 1,\ldots,m$. We permute the $n_{X} + n_{Y}$ observations at random, and then permute the $t$-statistic on the permuted data. For this permuted data, we know all the null hypotheses hold, and so the number of permuted $t$-statistics that exceed the threshold $c$ in absolute value provides an estimate for $E(V)$. We can improve this estimate by repeating the permutation process $B$ times, for some large number $B$, and averaging the results. This procedure is described in the algorithm below:

1. Select a threshold $c$, where $c > 0$.

2. For $j = 1,\ldots,m:

    (a) Compute $T^{(j)}$, the two-sample $t$-statistic for the null hypothesis $H_{0j}$ on the basis of the original data $x_{1}^{(j)},\ldots,x_{n_{X}}^{(j)}$ and $y_{1}^{(j)},\ldots,y_{n_{Y}}^{(j)}$.
    
    (b) For $b = 1,\ldots,B$, where $B$ is a large number (like $10,000$)
    
    - Permute the $n_{X}+ n_{Y}$ observations at random. Call the first $n_{X}$ observations $x_{1}^{*(j)},x_{2}^{*(j)},\ldots,x_{n_{X}}^{*(j)}$, and call the remaining observations $y_{1}^{*(j)},y_{2}^{*(j)},\ldots,y_{n_{Y}}^{*(j)}$.
    
    - Compute the T statistic on the permuted data $x_{1}^{*(j)},x_{2}^{*(j)},\ldots,x_{n_{X}}^{*(j)}$, and $y_{1}^{*(j)},y_{2}^{*(j)},\ldots,y_{n_{Y}}^{*(j)}$, and call the result $T^{(j),*b}$.
    
3. Compute $\displaystyle R = \sum_{i = 1}^{m} 1_{(|T_{j}| \geq c)}$

4. Compute $\displaystyle \hat{V} = \frac{\sum_{b = 1}^{B} \sum_{j = 1}^{m} 1_{(|T^{(j),*b}| \geq c)}}{B}$

5. The estimated FDR associated with the threshold $c$ is $\displaystyle \frac{\hat{V}}{R}$

In general, there are two settings in which a re-sampling approach is particularly useful:

1. Perhaps no theoretical null distribution is available. This may be the case if you are testing an unusual null hypothesis $H_{0}$, or using an unusual test statistic $T$.

2. Perhaps a theoretical null distribution *is* available, but the assumptions required for its validity do not hold. **For example, you don't have a large number of observations.** 

In many real-world settings, re-sampling is a powerful tool for hypothesis testing when no out-of-the-box hypothesis tests are available, or key assumptions are violated.